In [1]:
import torch
import os
from torchvision import transforms,datasets
from torch.utils.data import DataLoader, random_split

In [2]:
from data_generator import FingerPrintDataset, FingerPrintDataset_rgb

In [3]:
data_root= '/Users/ayush/projects/my_pytorch/data_trial/'
dataset = FingerPrintDataset(data_root,transform=transforms.Compose([
                                 transforms.ToTensor()
                                 
                             ]))
dataset_rgb = FingerPrintDataset_rgb(data_root,transform=transforms.Compose([
                                 transforms.ToTensor()
                                 
                             ]))

/Users/ayush/projects/my_pytorch/data_trial/train
/Users/ayush/projects/my_pytorch/data_trial/train


In [4]:

train_dataset = FingerPrintDataset(data_root, train=True,
                             transform=transforms.Compose([
                                 transforms.ToTensor()]))
test_dataset = FingerPrintDataset(data_root, train=False,
                            transform=transforms.Compose([
                                transforms.ToTensor()]))




train_dataset_rgb = FingerPrintDataset_rgb(data_root, train=True,
                             transform=transforms.Compose([
                                 transforms.ToTensor()]))
test_dataset_rgb = FingerPrintDataset_rgb(data_root, train=False,
                            transform=transforms.Compose([
                                transforms.ToTensor()]))

/Users/ayush/projects/my_pytorch/data_trial/train
/Users/ayush/projects/my_pytorch/data_trial/test
/Users/ayush/projects/my_pytorch/data_trial/train
/Users/ayush/projects/my_pytorch/data_trial/test


In [5]:
i,l =train_dataset_rgb[4]
i.shape

torch.Size([3, 512, 512])

In [6]:
train_classes_set=set()
test_classes_set=set()
for x in train_dataset:
    train_classes_set.add(x[1])
for x in test_dataset:
    test_classes_set.add(x[1])

train_classes = len(train_classes_set)
test_classes = len(test_classes_set)
train_classes_samples=2
test_classes_samples=2
print(train_classes,test_classes)

24 16


In [7]:
from networks import EmbeddingNet, TripletNet
from losses import TripletLoss
from torch.optim import lr_scheduler
import torch.optim as optim
from trainer import fit
cuda = torch.cuda.is_available()

In [8]:

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

mnist_classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9','10','11','12','13','14','15']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
              '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
              '#bcbd22', '#17becf','#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
              '#bcbd22']
def plot_embeddings(embeddings, targets, xlim=None, ylim=None):
    plt.figure(figsize=(10,10))
    for i in range(15):
        inds = np.where(targets==i)[0]
        print(inds,i)
        plt.scatter(embeddings[inds,0], embeddings[inds,1], alpha=0.5, color=colors[i])
    if xlim:
        plt.xlim(xlim[0], xlim[1])
    if ylim:
        plt.ylim(ylim[0], ylim[1])
    plt.legend(mnist_classes)

def extract_embeddings(dataloader, model):
    with torch.no_grad():
        model.eval()
        embeddings = np.zeros((len(dataloader.dataset), 2))
        labels = np.zeros(len(dataloader.dataset))
        k = 0
        for images, target in dataloader:
            if cuda:
                images = images.cuda()
            embeddings[k:k+len(images)] = model.get_embedding(images).data.cpu().numpy()
            labels[k:k+len(images)] = target.numpy()
            k += len(images)
    return embeddings, labels



In [9]:
from datasets import BalancedBatchSampler

train_batch_sampler = BalancedBatchSampler(train_dataset.train_labels, n_classes=train_classes, n_samples=train_classes_samples)
test_batch_sampler = BalancedBatchSampler(test_dataset.test_labels, n_classes=test_classes, n_samples=train_classes_samples)

train_batch_sampler_rgb = BalancedBatchSampler(train_dataset_rgb.train_labels, n_classes=train_classes, n_samples=train_classes_samples)
test_batch_sampler_rgb = BalancedBatchSampler(test_dataset_rgb.test_labels, n_classes=test_classes, n_samples=train_classes_samples)

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

online_train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_batch_sampler, **kwargs)
online_test_loader = torch.utils.data.DataLoader(test_dataset, batch_sampler=test_batch_sampler, **kwargs)


online_train_loader_rgb = torch.utils.data.DataLoader(train_dataset_rgb, batch_sampler=train_batch_sampler, **kwargs)
online_test_loader_rgb = torch.utils.data.DataLoader(test_dataset_rgb, batch_sampler=test_batch_sampler, **kwargs)

/Users/ayush/projects/my_pytorch/data_generator.py:33: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/Users/ayush/projects/my_pytorch/data_generator.py:38: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
/Users/ayush/projects/my_pytorch/data_generator.py:103: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/Users/ayush/projects/my_pytorch/data_generator.py:108: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [10]:
# Set up the network and training parameters
import torchvision
from torch import nn
from networks import EmbeddingNet
from losses import OnlineTripletLoss
from utils import AllTripletSelector,HardestNegativeTripletSelector, RandomNegativeTripletSelector, SemihardNegativeTripletSelector # Strategies for selecting triplets within a minibatch
from metrics import AverageNonzeroTripletsMetric

margin = 1.
embedding_net = EmbeddingNet()
#model = embedding_net
model=torchvision.models.squeezenet1_1(pretrained=True)
model.classifier[1] = nn.Conv2d(512, 256, kernel_size=(1,1), stride=(1,1))
for param in model.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True



In [11]:
#model

In [15]:
if cuda:
    model.cuda()
loss_fn = OnlineTripletLoss(margin, SemihardNegativeTripletSelector(margin))
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 15
log_interval = 1

In [ ]:
#fit(online_train_loader, online_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[AverageNonzeroTripletsMetric()])

##############################################################################################
optimizer=torch.optim.AdamW(model.classifier.parameters())
fit(online_train_loader_rgb, online_test_loader_rgb, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[AverageNonzeroTripletsMetric()])
for param in model.parameters():
    param.requires_grad = True
optimizer=torch.optim.AdamW(model.parameters(),lr=0.00001)
fit(online_train_loader_rgb, online_test_loader_rgb, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[AverageNonzeroTripletsMetric()])



0 torch.Size([44, 3, 512, 512]) torch.Size([44])
Train: [0/113 (0%)]	Loss: 0.585611	Average nonzero triplets: 13.0
1 torch.Size([44, 3, 512, 512]) torch.Size([44])
Train: [3/113 (50%)]	Loss: 0.533194	Average nonzero triplets: 12.5
Epoch: 1/15. Train set: Average loss: 0.5594	Average nonzero triplets: 12.5
Epoch: 1/15. Validation set: Average loss: 0.4888	Average nonzero triplets: 10.333333333333334
0 torch.Size([44, 3, 512, 512]) torch.Size([44])
Train: [0/113 (0%)]	Loss: 0.539685	Average nonzero triplets: 14.0
1 torch.Size([44, 3, 512, 512]) torch.Size([44])
Train: [3/113 (50%)]	Loss: 0.619927	Average nonzero triplets: 14.5
model saved as ./saved_model/model_train_loss_0.5798059403896332_epoch_1
Epoch: 2/15. Train set: Average loss: 0.5798	Average nonzero triplets: 14.5
Epoch: 2/15. Validation set: Average loss: 0.5292	Average nonzero triplets: 10.0
0 torch.Size([44, 3, 512, 512]) torch.Size([44])
Train: [0/113 (0%)]	Loss: 0.642347	Average nonzero triplets: 10.0
1 torch.Size([44, 3, 5

In [21]:
for a,b in enumerate(online_train_loader_rgb):
    print(b)

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 1 and 3 in dimension 1 at /Users/distiller/project/conda/conda-bld/pytorch_1570710797334/work/aten/src/TH/generic/THTensor.cpp:689

In [10]:
'''
import numpy as np
train_embeddings_otl, train_labels_otl = extract_embeddings(online_train_loader, model)
plot_embeddings(train_embeddings_otl, train_labels_otl)
val_embeddings_otl, val_labels_otl = extract_embeddings(online_test_loader, model)
plot_embeddings(val_embeddings_otl, val_labels_otl)
'''

'\nimport numpy as np\ntrain_embeddings_otl, train_labels_otl = extract_embeddings(online_train_loader, model)\nplot_embeddings(train_embeddings_otl, train_labels_otl)\nval_embeddings_otl, val_labels_otl = extract_embeddings(online_test_loader, model)\nplot_embeddings(val_embeddings_otl, val_labels_otl)\n'

In [11]:
from trainer import final_test_epoch

In [12]:
mm=final_test_epoch('/Users/ayush/projects/my_pytorch/probe' ,'/Users/ayush/projects/my_pytorch/gallery',model,metrics=[AverageNonzeroTripletsMetric()],transform=transforms.Compose([
                                 transforms.ToTensor()]))